<a href="https://colab.research.google.com/github/anushka-code/Code-Smell-Classification/blob/main/Code_Smell_Classification_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Structural Branch of the NN-Architecture : CNN 1-D for Structural Object Oriented Metric Feature Extraction

##Code Smells Targetted: 


1.   Long Parameters List
2.   Switch Statements







###Install Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Convolution1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.utils import np_utils
import sklearn
from sklearn.preprocessing import MinMaxScaler
import imblearn
from imblearn.over_sampling import SMOTE

###Mounting Google Drive

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
    
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

###Dataset Loader

In [3]:
def DataLoader(link, name_of_file):
  id = link.split("/")[-2]
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile(name_of_file)
  dataframe = pd.read_csv(name_of_file)
  return dataframe

link1 = 'https://drive.google.com/file/d/1EfbAqgr7i9h4yFwEoU3igG34Gt48l6WT/view?usp=sharing'
link2 = 'https://drive.google.com/file/d/1Ya1OMWsz1yyXAaZheIck-roX0M9UWiqg/view?usp=sharing'
name1 = 'long_parameters_list_structural.csv'
name2 = 'switch_statements_structural.csv'

df_lp = DataLoader(link1, name1)
df_ss = DataLoader(link2, name2)

Long Parameters List Dataset

In [4]:
df_lp.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,MaMCL_method,NOAV_method,LAA_method,FANOUT_method,CFNAMM_method,ATLD_method,CLNAMM_method,CINT_method,MeMCL_method,CDISP_method,NOII_type,NOAM_type,NOCS_type,NOM_type,NMO_type,ATFD_type,FANOUT_type,NOMNAMM_type,NOA_type,NIM_type,DIT_type,LOC_type,LOCNAMM_type,CFNAMM_type,TCC_type,NOPA_type,CBO_type,RFC_type,NOC_type,WMC_type,LCOM5_type,WOC_type,WMCNAMM_type,AMW_type,AMWNAMM_type,NOCS_package,NOMNAMM_package,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_long_parameters_list
0,0,0,0,0,0,2,8,2,0,0,0,6,1,0,2,3,0,0,0,0.0,0,7,2,16,?,2,3,9,12,?,4,221,172,406,0.177778,1,9,34,1,29,0.961111,?,18,1.812500,2.000000,6,9,0,229,16,43,849,78,5788,4778,65687,False
1,0,0,0,0,0,1,3,1,0,0,0,1,1,0,0,1,0,0,0,0.0,1,0,0,3,1,0,0,3,1,0,1,16,16,0,1.000000,0,0,3,0,3,0.500000,1,3,1.000000,1.000000,38,198,0,4223,240,43,849,78,5788,4778,65687,False
2,1,0,0,0,0,1,2,1,0,1,0,1,0,1,3,0,0,1,0,1.0,1,29,18,123,17,50,138,94,108,77,2,1957,1669,51801,0.012613,87,145,571,0,273,0.992107,0.404145078,234,2.219512,2.489362,38,198,0,4223,240,43,849,78,5788,4778,65687,False
3,0,5,0,0,9,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1,21,0,92,?,0,0,71,5,?,?,288,206,0,0.000000,5,15,92,0,92,1.010989,0,71,1.000000,1.000000,34,523,11,6037,728,43,849,78,5788,4778,65687,False
4,1,0,1,1,0,1,3,1,0,1,0,2,0,1,0,0,0,1,0,1.0,1,0,0,3,0,1,1,3,0,0,1,12,12,0,0.000000,0,1,4,0,3,0.000000,1,3,1.000000,1.000000,34,523,11,6037,728,43,849,78,5788,4778,65687,False


Switch Statements Dataset

In [5]:
df_ss.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,MaMCL_method,NOAV_method,LAA_method,FANOUT_method,CFNAMM_method,ATLD_method,CLNAMM_method,CINT_method,MeMCL_method,CDISP_method,NOII_type,NOAM_type,NOCS_type,NOM_type,NMO_type,ATFD_type,FANOUT_type,NOMNAMM_type,NOA_type,NIM_type,DIT_type,LOC_type,LOCNAMM_type,CFNAMM_type,TCC_type,NOPA_type,CBO_type,RFC_type,NOC_type,WMC_type,LCOM5_type,WOC_type,WMCNAMM_type,AMW_type,AMWNAMM_type,NOCS_package,NOMNAMM_package,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_switch_statements
0,1,2,0,0,2,1,5,1,0,1,0,1,0,1,4,0,0,2,0,0.5,0,2,0,7,?,1,3,5,2,?,3,53,39,55,0.100000,0,4,13,2,8,0.916667,?,6,1.142857,1.200000,7,47,3,439,53,43,849,78,5788,4778,65687,False
1,0,0,0,0,0,2,6,3,0,0,0,1,1,0,1,1,0,0,0,0.0,0,8,0,30,12,4,10,22,8,111,3,377,310,1214,0.057143,0,32,208,0,61,0.943966,0.681818182,51,2.033333,2.318182,38,198,0,4223,240,43,849,78,5788,4778,65687,False
2,2,0,0,0,0,1,3,1,0,2,0,1,0,1,2,0,0,1,0,1.0,1,0,0,1,0,0,1,1,0,0,1,3,3,4,1.000000,0,3,2,?,1,0.000000,?,1,1.000000,1.000000,38,198,0,4223,240,43,849,78,5788,4778,65687,False
3,0,0,0,0,0,2,3,1,0,0,0,0,0,1,2,0,0,1,0,1.0,1,0,0,1,0,0,1,1,0,0,1,5,5,4,1.000000,0,1,2,?,1,0.000000,?,1,1.000000,1.000000,18,102,0,1212,120,43,849,78,5788,4778,65687,False
4,0,0,0,0,0,1,3,1,0,0,0,6,0,0,0,0,0,0,0,0.0,0,2,1,16,0,7,10,14,6,8,3,197,179,640,0.305556,0,10,41,0,31,0.855556,1,28,1.937500,2.000000,4,27,0,516,29,43,849,78,5788,4778,65687,False


### Data Pre-Processing 

In [6]:
def PrePro(last_column, dataframe):
  dataframe.rename(columns = {last_column :'is_code_smell'}, inplace = True)
  dataframe['is_code_smell'] = dataframe["is_code_smell"].astype(int)
  Y_part = dataframe.iloc[:,-1:]
  X_part = dataframe.iloc[:,:56]
  X_part = X_part.replace(to_replace =["?"], value = np.nan)
  X_part = X_part.astype(float)
  return X_part,Y_part

X_lp, Y_lp = PrePro('is_long_parameters_list',df_lp)
X_ss, Y_ss = PrePro('is_switch_statements',df_ss)

In [7]:
def MeanforNaN(dataframe):  
  column_means = dataframe.mean()
  dataframe = dataframe.fillna(column_means)
  return dataframe

X_lp = MeanforNaN(X_lp)
X_ss = MeanforNaN(X_ss)

In [8]:
def ConCat(df1,df2):
  code_smells = [df1,df2]
  joint = pd.concat(code_smells)
  return joint

X_train = ConCat(X_lp,X_ss)
Y_train = ConCat(Y_lp,Y_ss)

###Data Normalization

In [9]:
def Normalize(dataframe):
  scaler = MinMaxScaler()
  model = scaler.fit(dataframe)
  scaled_data = model.transform(dataframe)
  return scaled_data

X_sample = Normalize(X_train)

###SMOTE's Algo : Oversampling smelly samples (minority class)



In [10]:
Y_train.value_counts() #1/3rd are smelly, 2/3rd are non smelly 

is_code_smell
0                573
1                267
dtype: int64

In [11]:
Y_sample = Y_train.to_numpy(dtype='int64', copy='True')

In [12]:
def Oversample(X_data,Y_data):

  sm = SMOTE(random_state = 2)
  X_train_res, Y_train_res = sm.fit_resample(X_data, Y_data.ravel())
  return X_train_res, Y_train_res

X_new, Y_new = Oversample(X_sample,Y_sample)

### 1-D CNN Implementation

In [14]:
X_train_final = X_new.reshape((1146,56,1))

In [17]:
  model = Sequential()
  model.add(Convolution1D(filters=128, kernel_size=3, activation='tanh', input_shape=(56,1)))
  model.add(Convolution1D(filters=128, kernel_size=3, activation='tanh'))
  model.add(Convolution1D(filters=128, kernel_size=5, activation='tanh'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 54, 128)           512       
                                                                 
 conv1d_1 (Conv1D)           (None, 52, 128)           49280     
                                                                 
 conv1d_2 (Conv1D)           (None, 48, 128)           82048     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 24, 128)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 1)                 3073      
                                                        

In [19]:
hist = model.fit(X_train_final,Y_new,epochs=30,shuffle=True,batch_size=50,validation_split=0.20) 

Epoch 1/30
19/19 [==============================] - 5s 130ms/step - loss: 0.6316 - accuracy: 0.6769 - val_loss: 0.8063 - val_accuracy: 0.4739
Epoch 2/30
19/19 [==============================] - 3s 147ms/step - loss: 0.4980 - accuracy: 0.7609 - val_loss: 0.4791 - val_accuracy: 0.6957
Epoch 3/30
19/19 [==============================] - 2s 113ms/step - loss: 0.4404 - accuracy: 0.8013 - val_loss: 0.3898 - val_accuracy: 0.8261
Epoch 4/30
19/19 [==============================] - 2s 111ms/step - loss: 0.3980 - accuracy: 0.8362 - val_loss: 0.6090 - val_accuracy: 0.6217
Epoch 5/30
19/19 [==============================] - 2s 96ms/step - loss: 0.3657 - accuracy: 0.8308 - val_loss: 0.2639 - val_accuracy: 0.8870
Epoch 6/30
19/19 [==============================] - 2s 78ms/step - loss: 0.3594 - accuracy: 0.8493 - val_loss: 0.6656 - val_accuracy: 0.6522
Epoch 7/30
19/19 [==============================] - 1s 60ms/step - loss: 0.3363 - accuracy: 0.8395 - val_loss: 0.2893 - val_accuracy: 0.8739
Epoch 8/3